In [ ]:
%pip install transformers

In [ ]:
%pip install torch

In [ ]:
%pip install hf_xet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 2.9 MB/s eta 0:00:00a 0:00:01


In [39]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForSeq2SeqLM
import torch
import torch.nn.functional as F
import re
from transformers import pipeline

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

In [26]:
nli_model_name = "roberta-large-mnli"
nli_tokenizer = AutoTokenizer.from_pretrained(nli_model_name)
nli_model = AutoModelForSequenceClassification.from_pretrained(nli_model_name).to(device)
nli_model.eval()

Some weights of the model checkpoint at roberta-large-mnli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=Tru

In [27]:
def does_passage_support_quote(quote, passage):
    inputs = nli_tokenizer(passage, quote, return_tensors="pt", truncation=True).to(device)
    with torch.no_grad():
        logits = nli_model(**inputs).logits
        probs = F.softmax(logits, dim=1)

    support_prob = probs[0][2].item()
    contradict_prob = probs[0][0].item()
    unclear_prob = probs[0][1].item()

    print(f"Supports: {support_prob:.2f}, Unclear: {unclear_prob:.2f}, Contradicts: {contradict_prob:.2f}")

    if support_prob > contradict_prob:
        return "supports"
    else:
        return "contradicts"

In [40]:
qa_pipeline = pipeline("question-answering", model="distilbert-base-cased-distilled-squad")

def extract_relevant_quote(claim, passage, label):
    if label.lower() == "supports":
        question = f"What sentence in the passage supports the claim: {claim}"
    elif label.lower() == "contradicts":
        question = f"What sentence in the passage contradicts the claim: {claim}"
    else:
        return "No supporting or contradicting quote found."

    result = qa_pipeline(question=question, context=passage)
    answer = result["answer"]

    sentences = re.split(r'(?<=[.!?]) +', passage)
    for sentence in sentences:
        if answer.strip() in sentence:
            return sentence.strip()

    return answer

Device set to use mps:0


In [ ]:
# quote = "Climate change is caused by human activities."
# passage = (
#     "Climate scientists have gathered overwhelming evidence that human activities, "
#     "particularly the burning of fossil fuels like coal, oil, and gas, are the primary drivers of recent climate change. "
#     "The resulting increase in greenhouse gases, such as carbon dioxide and methane, has led to a warming of the Earth's atmosphere, "
#     "oceans, and land surfaces. Numerous studies conducted over the past decades consistently link industrial emissions to rising "
#     "global temperatures, melting ice caps, more frequent extreme weather events, and sea-level rise. In 2021, the Intergovernmental "
#     "Panel on Climate Change (IPCC) declared that it is 'unequivocal' that human influence has warmed the atmosphere, ocean, and land. "
#     "These findings are based on a combination of observational data, climate modeling, and attribution studies."
# )

# relation = does_passage_support_quote(quote, passage)
# print(f"\nRelation: {relation}")

# if relation in ["supports", "contradicts"]:
#     extracted_quote = extract_relevant_quote(quote, passage, relation)
#     print(f"\nExtracted {relation} quote:\n{extracted_quote}")
# else:
#     print("\nNo clear support or contradiction; skipping quote extraction.")

In [ ]:
quote = input_text
passage = (
    "Climate scientists have gathered overwhelming evidence that human activities, "
    "particularly the burning of fossil fuels like coal, oil, and gas, are the primary drivers of recent climate change. "
    "The resulting increase in greenhouse gases, such as carbon dioxide and methane, has led to a warming of the Earth's atmosphere, "
    "oceans, and land surfaces. Numerous studies conducted over the past decades consistently link industrial emissions to rising "
    "global temperatures, melting ice caps, more frequent extreme weather events, and sea-level rise. In 2021, the Intergovernmental "
    "Panel on Climate Change (IPCC) declared that it is 'unequivocal' that human influence has warmed the atmosphere, ocean, and land. "
    "These findings are based on a combination of observational data, climate modeling, and attribution studies."
)

relation = does_passage_support_quote(quote, passage)
print(f"\nRelation: {relation}")

if relation in ["supports", "contradicts"]:
    extracted_quote = extract_relevant_quote(quote, passage, relation)
    print(f"\nExtracted {relation} quote:\n{extracted_quote}")
else:
    print("\nNo clear support or contradiction; skipping quote extraction.")